## Some testing and analysis of the new `Snapshot` implementation

In [1]:
import numpy as np
import mdtraj as md
import pandas as pd
import simtk.openmm as mm
from simtk.openmm import app
from simtk import unit
 
from openpathsampling.visualize import PathTreeBuilder
from IPython.display import SVG

import openpathsampling as paths

In [2]:
from IPython.display import Markdown

def code_to_md(snapshot_class):
    md = '```py\n'
    for f, s in snapshot_class.__features__.debug.iteritems():
        md += s
        md += '\n\n'
    md += '```'

    return md

### Show the generated source code

In [3]:
EmptySnap = paths.engines.snapshot.SnapshotFactory('no', [], 'Empty', use_lazy_reversed=False)

In [4]:
import openpathsampling.engines.features as features

In [5]:
@features.base.attach_features([
    features.velocities,
    features.coordinates,
    features.box_vectors,
    features.topology
])
class A(paths.BaseSnapshot):
    def copy(self):
        pass

class B(paths.BaseSnapshot):
    pass

In [6]:
a = A()
b = a.copy()

In [7]:
Markdown(code_to_md(A))

```py
def create_empty(self):
    this = cls.__new__(cls)
    this._reversed = None
    return this

@staticmethod
def init_copy(self, velocities=None, coordinates=None, box_vectors=None, topology=None):
    this._reversed = None
    self.topology = topology
    np.copyto(self.velocities, velocities)
    np.copyto(self.coordinates, coordinates)
    np.copyto(self.box_vectors, box_vectors)

def create_reversed(self):
    this = cls.__new__(cls)
    this._reversed = self
    this.coordinates = self.coordinates
    this.box_vectors = self.box_vectors
    this.topology = self.topology
    this.velocities = - self.velocities
    return this

def copy_to(self, target):
    target._reversed = None
    target.topology = self.topology
    np.copyto(target.velocities, self.velocities)
    np.copyto(target.coordinates, self.coordinates)
    np.copyto(target.box_vectors, self.box_vectors)

def init_empty(self):
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._reversed = None
    this.topology = self.topology
    this.velocities = self.velocities.copy()
    this.coordinates = self.coordinates.copy()
    this.box_vectors = self.box_vectors.copy()
    return this

def __init__(self, velocities=None, coordinates=None, box_vectors=None, topology=None):
    self._reversed = None
    self.velocities = velocities
    self.coordinates = coordinates
    self.box_vectors = box_vectors
    self.topology = topology

```

In [8]:
print hasattr(A, 'copy')
print 'copy' in A.__dict__

True
True


In [9]:
Markdown(code_to_md(EmptySnap))

```py
def create_empty(self):
    this = cls.__new__(cls)
    this._reversed = None
    return this

@staticmethod
def init_copy(self):
    this._reversed = None

def create_reversed(self):
    this = cls.__new__(cls)
    this._reversed = self
    return this

def copy_to(self, target):
    target._reversed = None

def init_empty(self):
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._reversed = None
    return this

def __init__(self):
    self._reversed = None

```

In [10]:
SuperSnap = paths.engines.snapshot.SnapshotFactory(
    'my', [
        paths.engines.features.coordinates,
        paths.engines.features.box_vectors,
        paths.engines.features.velocities
    ], 'No desc', use_lazy_reversed=False)

In [32]:
SuperSnap.__dict__

<dictproxy {'__abstractmethods__': frozenset(),
 '__doc__': 'No desc\n\nAttributes\n----------\ncoordinates : numpy.ndarray, shape=(atoms, 3), dtype=numpy.float32\n    atomic coordinates\n\nvelocities : numpy.ndarray, shape=(atoms, 3), dtype=numpy.float32\n    atomic velocities\n',
 '__features__': __features__(classes=[<module 'openpathsampling.engines.features.coordinates' from '/Users/jan-hendrikprinz/Studium/git/openpathsampling/openpathsampling/engines/features/coordinates.pyc'>, <module 'openpathsampling.engines.features.box_vectors' from '/Users/jan-hendrikprinz/Studium/git/openpathsampling/openpathsampling/engines/features/box_vectors.pyc'>, <module 'openpathsampling.engines.features.velocities' from '/Users/jan-hendrikprinz/Studium/git/openpathsampling/openpathsampling/engines/features/velocities.pyc'>], variables=['coordinates', 'box_vectors', 'velocities'], properties=['xyz'], functions=[], required=[], lazy=[], numpy=['coordinates', 'box_vectors', 'velocities'], reversal=['

In [11]:
Markdown(code_to_md(SuperSnap))

```py
def create_empty(self):
    this = cls.__new__(cls)
    this._reversed = None
    return this

@staticmethod
def init_copy(self, coordinates=None, box_vectors=None, velocities=None):
    this._reversed = None
    np.copyto(self.coordinates, coordinates)
    np.copyto(self.box_vectors, box_vectors)
    np.copyto(self.velocities, velocities)

def create_reversed(self):
    this = cls.__new__(cls)
    this._reversed = self
    this.coordinates = self.coordinates
    this.box_vectors = self.box_vectors
    this.velocities = - self.velocities
    return this

def copy_to(self, target):
    target._reversed = None
    np.copyto(target.coordinates, self.coordinates)
    np.copyto(target.box_vectors, self.box_vectors)
    np.copyto(target.velocities, self.velocities)

def init_empty(self):
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._reversed = None
    this.coordinates = self.coordinates.copy()
    this.box_vectors = self.box_vectors.copy()
    this.velocities = self.velocities.copy()
    return this

def __init__(self, coordinates=None, box_vectors=None, velocities=None):
    self._reversed = None
    self.coordinates = coordinates
    self.box_vectors = box_vectors
    self.velocities = velocities

```

In [12]:
MegaSnap = paths.engines.snapshot.SnapshotFactory(
    'mega', [
        paths.engines.features.statics,
        paths.engines.features.kinetics,
        paths.engines.features.engine
    ], 'Long desc', use_lazy_reversed=False)

In [13]:
Markdown(code_to_md(MegaSnap))

```py
def create_empty(self):
    this = cls.__new__(cls)
    this._lazy = {}
    this._reversed = None
    return this

@staticmethod
def init_copy(self, statics=None, kinetics=None, is_reversed=False, engine=None):
    this._lazy = {
       cls.statics : statics,
       cls.kinetics : kinetics,
    }
    this._reversed = None
    self.is_reversed = is_reversed
    self.engine = engine

def create_reversed(self):
    this = cls.__new__(cls)
    this._lazy = {
       cls.statics : self._lazy[cls.statics],
       cls.kinetics : self._lazy[cls.kinetics],
    }
    this._reversed = self
    this.engine = self.engine
    this.is_reversed = not self.is_reversed
    return this

def copy_to(self, target):
    target._lazy = {
       cls.statics : self._lazy[cls.statics],
       cls.kinetics : self._lazy[cls.kinetics],
    }
    target._reversed = None
    target.is_reversed = self.is_reversed
    target.engine = self.engine

def init_empty(self):
    self._lazy = {}
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._lazy = {
       cls.statics : self._lazy[cls.statics],
       cls.kinetics : self._lazy[cls.kinetics],
    }
    this._reversed = None
    this.is_reversed = self.is_reversed
    this.engine = self.engine
    return this

def __init__(self, statics=None, kinetics=None, is_reversed=False, engine=None):
    self._lazy = {
       cls.statics : statics,
       cls.kinetics : kinetics,
    }
    self._reversed = None
    self.is_reversed = is_reversed
    self.engine = engine

```

### Check the timing

In [14]:
arr = np.array(range(200000))
brr = np.array(range(200000))

In [15]:
a = SuperSnap(arr, arr, arr)

In [16]:
%%timeit
a = SuperSnap(arr, arr, arr)

The slowest run took 7.52 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 665 ns per loop


In [17]:
%%timeit
b = SuperSnap()

The slowest run took 10.02 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 595 ns per loop


In [18]:
%%timeit
b = SuperSnap(coordinates=arr, box_vectors=arr, velocities=arr)

The slowest run took 7.25 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 822 ns per loop


In [19]:
x = EmptySnap()

In [20]:
%%timeit
x = EmptySnap()

The slowest run took 7.95 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 390 ns per loop


In [21]:
%%timeit
x.reversed

The slowest run took 31.94 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 284 ns per loop


In [22]:
%%timeit
a.reversed

The slowest run took 978.08 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 295 ns per loop


In [23]:
assert(x.reversed.reversed is x)
assert(a.reversed.reversed is a)

In [24]:
%%timeit
EmptySnap.__init__(x)

The slowest run took 17.92 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 279 ns per loop


In [25]:
b = SuperSnap()

In [39]:
%%timeit
brr.copy()

10000 loops, best of 3: 86.9 µs per loop


In [26]:
%%timeit
SuperSnap.__init__(b, arr, arr, arr)

The slowest run took 11.94 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 499 ns per loop


In [48]:
%%timeit
c = a.copy()

1000 loops, best of 3: 307 µs per loop


In [46]:
b = SuperSnap(brr, brr, brr)

In [47]:
%%timeit
a.copy_to(b)

1000 loops, best of 3: 249 µs per loop


In [49]:
Markdown(code_to_md(a))

```py
def create_empty(self):
    this = cls.__new__(cls)
    this._reversed = None
    return this

@staticmethod
def init_copy(self, coordinates=None, box_vectors=None, velocities=None):
    this._reversed = None
    np.copyto(self.coordinates, coordinates)
    np.copyto(self.box_vectors, box_vectors)
    np.copyto(self.velocities, velocities)

def create_reversed(self):
    this = cls.__new__(cls)
    this._reversed = self
    this.coordinates = self.coordinates
    this.box_vectors = self.box_vectors
    this.velocities = - self.velocities
    return this

def copy_to(self, target):
    target._reversed = None
    np.copyto(target.coordinates, self.coordinates)
    np.copyto(target.box_vectors, self.box_vectors)
    np.copyto(target.velocities, self.velocities)

def init_empty(self):
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._reversed = None
    this.coordinates = self.coordinates.copy()
    this.box_vectors = self.box_vectors.copy()
    this.velocities = self.velocities.copy()
    return this

def __init__(self, coordinates=None, box_vectors=None, velocities=None):
    self._reversed = None
    self.coordinates = coordinates
    self.box_vectors = box_vectors
    self.velocities = velocities

```

In [50]:
%%timeit
SuperSnap.init_copy(a, arr, arr, arr)

NameError: global name 'this' is not defined

In [25]:
%%timeit
SuperSnap.init_empty(a)

The slowest run took 11.33 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 252 ns per loop
